In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [ ]:
train_img_Path = '../input/plant-pathology-2021-fgvc8/train_images'

test_img_Path = '../input/plant-pathology-2021-fgvc8/test_images'

img_Path = '../input/resized-plant2021/img_sz_256'

train = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/train.csv')

sample_submission = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
print(f'Number of pictures in the training dataset: {train.shape[0]}\n')
print(f'Number of different labels: {len(train.labels.unique())}\n')
print(f'Labels: {train.labels.unique()}')

In [ ]:
train.head()

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(14,7))
b = sns.countplot(x='labels', data=train, order=sorted(train['labels'].unique()))
for item in b.get_xticklabels():
    item.set_rotation(90)
plt.title('Label Distribution', weight='bold')
plt.show()

In [ ]:
import cv2

In [ ]:
plt.figure(figsize=(20,40))
i=1
for idx,s in train.head(9).iterrows():
    img_path = os.path.join(img_Path,s['image'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig=plt.subplot(9,3,i)
    fig.imshow(img)
    fig.set_title(s['labels'])
    i+=1

In [ ]:
print(f'Number of pictures in the training dataset: {train.shape[0]}\n')
print(f'Number of different labels: {len(train.labels.unique())}\n')
print(f'Labels: {train.labels.unique()}')

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(14,7))
b = sns.countplot(x='labels', data=train, order=sorted(train['labels'].unique()))
for item in b.get_xticklabels():
    item.set_rotation(90)
plt.title('Label Distribution', weight='bold')
plt.show()

In [ ]:
CLASSES = train['labels'].unique().tolist()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   validation_split=0.25)

train_data = train_datagen.flow_from_dataframe(train,
                                              directory=img_Path,
                                              classes=CLASSES,
                                              x_col="image",
                                              y_col="labels",
                                              target_size=(150, 150),
                                              subset='training')

val_data = train_datagen.flow_from_dataframe(train,
                                            directory=img_Path,
                                            classes=CLASSES,
                                            x_col="image",
                                            y_col="labels",
                                            target_size=(150, 150),
                                            subset='validation')

In [ ]:
dict_classes = train_data.class_indices
dict_classes

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.applications import ResNet50V2, ResNet152V2, InceptionResNetV2
from tensorflow.keras.layers import Conv2D, Dropout,MaxPooling2D,Flatten,Dense

In [ ]:
base_ResNet152V2 = ResNet152V2(include_top = False, 
                         weights = '../input/resnet152v2notopimagenet/ResNet152V2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
model_ResNet = Sequential()
model_ResNet.add(base_ResNet152V2)
model_ResNet.add(Dense(12, activation=('softmax')))

model_ResNet.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_ResNet.summary()

# Training the CNN on the Train data and evaluating it on the val data
r = model_ResNet.fit(train_data, validation_data = val_data, epochs = 40, batch_size=32)

In [ ]:
model_history = r.history

plt.figure()
plt.plot(model_history['accuracy'])
plt.plot(model_history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.savefig('accuracy')
plt.show()

In [ ]:
plt.figure()
plt.plot(model_history['loss'])
plt.plot(model_history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.savefig('loss')
plt.show()

In [ ]:
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

test_data = train_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_dir,
                                    x_col="image",
                                    y_col=None,
                                    batch_size=32,
                                    seed=42,
                                    shuffle=False,
                                    class_mode=None,
                                    target_size=(150, 150))

In [ ]:
pred_resnet = model_ResNet.predict(test_data)
pred_resnet

In [ ]:
pred = (pred_resnet+pred_resnet+pred_resnet).tolist()

In [ ]:
for i in range(len(pred)):
    pred[i] = np.argmax(pred[i])

    
def get_key(val):
    for key, value in dict_classes.items():
        if val == value:
            return key
        

for i in range(len(pred)):
    pred[i] = get_key(pred[i])

In [ ]:
pred

In [ ]:
test_df['labels'] = pred
test_df

In [ ]:
test_df.to_csv('submission.csv',index=False)